In [82]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/data-storm-4/Data-dictionary.xlsx
/kaggle/input/data-storm-4/DataStorm4.0_KaggleProblemStatement.pdf
/kaggle/input/data-storm-4/Store-info.csv
/kaggle/input/data-storm-4/Testing-data.csv
/kaggle/input/data-storm-4/Historical-transaction-data.csv


In [83]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats import norm

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.cluster import KMeans
from sklearn.tree import DecisionTreeRegressor

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_squared_log_error

In [84]:
df = pd.read_csv("/kaggle/input/data-storm-4/Store-info.csv")
df2=pd.read_csv("/kaggle/input/data-storm-4/Historical-transaction-data.csv")

print(df.shape)
print(df2.shape)
df2.head()

(124, 3)
(473974, 7)


,item_description,transaction_date,invoice_id,customer_id,shop_id,item_price,quantity_sold
0,ORANGE BARLEY 1.5L,2021-12-11T00:00:00.000Z,147.0,BGXA,SHOP008,220,2
1,GINGER BEER 1.5L,2021-10-17T00:00:00.000Z,371.0,IA25,SHOP112,220,2
2,TONIC PET 500ML,2021-12-13T00:00:00.000Z,484.0,VN7V,SHOP008,160,2
3,CREAM SODA 1L,2021-12-13T00:00:00.000Z,484.0,VN7V,SHOP008,150,2
4,STRAWBERRY MILK 180ML,2021-10-23T00:00:00.000Z,1310.0,7S00,SHOP112,210,5


In [85]:
df2.columns
df2=df2[['shop_id', 'item_price', 'quantity_sold']].copy()
df2.head()

,shop_id,item_price,quantity_sold
0,SHOP008,220,2
1,SHOP112,220,2
2,SHOP008,160,2
3,SHOP008,150,2
4,SHOP112,210,5


In [86]:
shop_area=pd.read_csv("/kaggle/input/data-storm-4/Store-info.csv")
shop_area.columns

Index(['shop_id', 'shop_area_sq_ft', 'shop_profile'], dtype='object')

In [87]:
df3=shop_area[['shop_id', 'shop_area_sq_ft']].copy()
# df3 = df3.rename(index = df3['shop_id']) ### RENAME
df2["money"] = df2['quantity_sold']*df2['item_price']
# df3 = df3.drop(['shop_id'],axis=1)
df2
# min(df2['shop_id'].value_counts())

,shop_id,item_price,quantity_sold,money
0,SHOP008,220,2,440
1,SHOP112,220,2,440
2,SHOP008,160,2,320
3,SHOP008,150,2,300
4,SHOP112,210,5,1050
...,...,...,...,...
473969,SHOP056,440,3,1320
473970,SHOP018,220,2,440
473971,SHOP077,210,4,840
473972,SHOP077,440,3,1320


In [88]:
def calculate_profit(key):
    return df2[df2["shop_id"] == key]['money'].sum()

calculate_profit("SHOP070")


1325070

In [89]:
df3["Profit"]=df3["shop_id"].map(calculate_profit)
df3["Quantity"]=df3["shop_id"].map(lambda key:df2[df2["shop_id"] == key]['quantity_sold'].sum())

# df3["profit"]=np.zeros(len(df3),int)
# df3
# for transaction in range(len(df2)):

#     store = df2['shop_id'][transaction]
#     qty = df2['quantity_sold'][transaction]
#     pr = df2['item_price'][transaction]
#     df3["profit"][store] = df3["profit"][store]+pr*qty
#     if(transaction%1000 == 0):
#         print(transaction)

df3

,shop_id,shop_area_sq_ft,Profit,Quantity
0,SHOP047,528,842960,3288
1,SHOP009,676,1970870,8411
2,SHOP083,676,1691985,6644
3,SHOP117,676,2325980,8073
4,SHOP042,676,1340215,5844
...,...,...,...,...
119,SHOP087,527,1128935,3750
120,SHOP050,411,2399125,7668
121,SHOP061,699,2110800,6053
122,SHOP056,597,3023805,11163
